# main ideas
1. use txt files to store all necessary information
2. build a parser that reads my lines dynamically
3. build a cv builder that decides what elements are displayed

# temporary use:
1. use tags (#) to indicate all categories (e.g. what section you want it to be, e.g. "education") and abilities (e.g. "quant") involved so the final output, a pd.DataFrame() can be used to organize it sum experiences
2. to describe the details of the job, use the PI:: Dr. Owen Lo syntax -- to establish a new column
3. identation doesn't matter, but still, try to make it readable


In [1]:
test_entry = """
! variable definitions
! define synonyms this way
scholarship_and_awards = awards* = scholarship
quantitative_analysis* = quant
! define what is a component of what
quantitative_anlysis =~ R + Python


@ University Affiliated Award 2020
    2020
    institution:: UBC //this is a comment too
    $16000
    selection_ratio:: only 100 people per year
    #scholarship_and_awards

@ Research_Assistant
    201509-201806
    Job_Description::   quantitative analysis
        completed 100% of the tasks on time.
    PI:: Dr. Owen Lo
    #working_experience
    #academic
    #quantitative_analysis
    #R #Python #Nvivo

@ AnotherRAship
        $3000/hr 201509-201806
        quantitative analysis
        completed 100% of the tasks on time.
        other_specifications:: I don't know
#quant

"""

In [21]:
re.sub("\s+", " ", test_entry)

" @ University Affiliated Award 2020 institution:: UBC $16000 selection_ratio:: only 100 people per year #scholarship_and_awards @ Research_Assistant 201509-201806 Job_Description:: quantitative analysis completed 100% of the tasks on time. PI:: Dr. Owen Lo #working_experience #academic #quantitative_analysis #R #Python #Nvivo @ AnotherRAship $3000/hr 201509-201806 quantitative analysis completed 100% of the tasks on time. other_specifications:: I don't know #quant "

In [18]:
import re
import pandas as pd

def lex(text):
    for statement in text.split('\n'):
        stripped_statement = statement.strip()
        m_comment1 = re.search(r"!([^\r\n]+)", stripped_statement)
        m_comment2 = re.search(r"//([^\r\n]+)", stripped_statement)
        m_synonymous = re.match(r"(([^\r\n]+)\s*)(=\s*([^\r\n]+))+", stripped_statement)
        m_constructive = re.match(r"([^\r\n]+)\s*=~(\s*[^\r\n]+\s*\+)*(\s*[^\r\n]+\s*)", stripped_statement)
        m_title = re.match(r"@([^\r\n]+)", stripped_statement)
       
        if m_comment1:
            stripped_statement = re.sub(r"!([^\r\n]+)", r"", stripped_statement)
        
        elif m_comment2:
            stripped_statement = re.sub(r"//([^\r\n]+)", r"", stripped_statement)
                               
        elif m_constructive:
            y, x = m_constructive.group(0).split("=~")
            y = y.replace(" ","")
            yield "constructed", y
            for constructor in x.replace(" ", "").split("+"):
                yield "constructor", constructor
            stripped_statement = re.sub(r"([^\r\n]+)\s*=~(\s*[^\r\n]+\s*\+)*(\s*[^\r\n]+\s*)", r"", stripped_statement)
        
        elif m_synonymous:
            yield "synonymous", m_synonymous.group(0).replace(" ","").split("=")
            stripped_statement = re.sub(r"(([^\r\n]+)\s*)(=\s*([^\r\n]+))+", r"", stripped_statement)
                               
        elif m_title: 
            yield "title", m_title.group(1).strip()
            stripped_statement = re.sub(r"@([^\r\n]+)", r"", stripped_statement)
                   
        for t in stripped_statement.split():
            t = re.sub(r"\s+", " ",t)
            m_money = re.match(r"\$(\d+)[^\r\n]{0,6}(\w\wD)?", t)
            m_yyyymm = re.match(r"(\d{4,6})[-~]?(\d{4,6})?", t)
            m_tags = re.match(r"\#([^\r\n]+)", t)
            
            # self-defined categories
            m_other_specs = re.match(r"([^\r\n]+)::", t)
            if m_money: 
                yield "money", m_money.group(1).strip()
                if m_money.group(2) is not None: 
                    yield "money", m_money.group(2).strip()
            elif m_yyyymm:
                yield "start_ym", m_yyyymm.group(1).strip()
                if m_yyyymm.group(2) is not None:
                    yield "end_ym", m_yyyymm.group(2).strip()
                else:
                    yield "end_ym", "present"
            elif m_tags: yield "tag", m_tags.group(1).strip()
            elif m_other_specs: yield m_other_specs.group(1).strip(), ""
            else:
                yield "content", t.strip()

list(lex(test_entry))

[('synonymous', ['scholarship_and_awards', 'awards*', 'scholarship']),
 ('synonymous', ['quantitative_analysis*', 'quant']),
 ('constructed', 'quantitative_anlysis'),
 ('constructor', 'R'),
 ('constructor', 'Python'),
 ('title', 'University Affiliated Award 2020'),
 ('start_ym', '2020'),
 ('end_ym', 'present'),
 ('institution', ''),
 ('content', 'UBC'),
 ('money', '16000'),
 ('selection_ratio', ''),
 ('content', 'only'),
 ('content', '100'),
 ('content', 'people'),
 ('content', 'per'),
 ('content', 'year'),
 ('tag', 'scholarship_and_awards'),
 ('title', 'Research_Assistant'),
 ('start_ym', '201509'),
 ('end_ym', '201806'),
 ('Job_Description', ''),
 ('content', 'quantitative'),
 ('content', 'analysis'),
 ('content', 'completed'),
 ('content', '100%'),
 ('content', 'of'),
 ('content', 'the'),
 ('content', 'tasks'),
 ('content', 'on'),
 ('content', 'time.'),
 ('PI', ''),
 ('content', 'Dr.'),
 ('content', 'Owen'),
 ('content', 'Lo'),
 ('tag', 'working_experience'),
 ('tag', 'academic'),
 

In [3]:
def parse(lexemes):
    syntax_tree = {}
    title_count = -1
    current_title, current_subtitle, current_streak = "", "", ""
    predefined_labels = ["start_ym", "end_ym", "title", "money", "tag", "content"]
    for k, v in lexemes:
        if k == 'title':
            title_count += 1
            syntax_tree['entry'+str(title_count)] = {'title': v}
            current_title = v
            current_subtitle = ""
            content_streak = ""
        # the following two elifs filter out "synonymous" and "constructive" because they don't have following 'content'
        elif k not in predefined_labels: # user-specified subtitle 
            current_subtitle = k
            content_streak = ""
        elif k == "content":
            content_streak += v + " "
            if current_subtitle == "":
                syntax_tree['entry'+str(title_count)]['content'] = content_streak
            else:
                syntax_tree['entry'+str(title_count)][current_subtitle] = content_streak
        elif k == "tag":
            syntax_tree['entry'+str(title_count)][v] = 1 
        else:
            syntax_tree['entry'+str(title_count)][k] = v
    return syntax_tree

parse(lex(test_entry))

{'entry0': {'title': 'University Affiliated Award 2020',
  'start_ym': '2020',
  'end_ym': 'present',
  'institution': 'UBC ',
  'money': '16000',
  'selection_ratio': 'only 100 people per year ',
  'scholarship_and_awards': 1},
 'entry1': {'title': 'Research_Assistant',
  'start_ym': '201509',
  'end_ym': '201806',
  'Job_Description': 'quantitative analysis completed 100% of the tasks on time. ',
  'PI': 'Dr. Owen Lo ',
  'working_experience': 1,
  'academic': 1,
  'quantitative_analysis': 1,
  'R': 1,
  'Python': 1,
  'Nvivo': 1},
 'entry2': {'title': 'AnotherRAship',
  'money': '3000',
  'start_ym': '201509',
  'end_ym': '201806',
  'content': 'quantitative analysis completed 100% of the tasks on time. ',
  'other_specifications': "I don't know ",
  'quant': 1}}

In [4]:
def to_df(syntax_tree):
# maybe create some columns here, like duration?

    dfs = pd.DataFrame()
    for k, v in syntax_tree.items():
        if re.match(r"entry\d+", k): # skipping definitions
            new_df = {k:[v_] for k, v_ in v.items()}
            dfs = dfs.append(pd.DataFrame(new_df))
    return dfs.reset_index(drop=True)

to_df(parse(lex(test_entry)))

,title,start_ym,end_ym,institution,money,selection_ratio,scholarship_and_awards,Job_Description,PI,working_experience,academic,quantitative_analysis,R,Python,Nvivo,content,other_specifications,quant
0,University Affiliated Award 2020,2020,present,UBC,16000,only 100 people per year,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Research_Assistant,201509,201806,NaN,NaN,NaN,NaN,quantitative analysis completed 100% of the ta...,Dr. Owen Lo,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
2,AnotherRAship,201509,201806,NaN,3000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,quantitative analysis completed 100% of the ta...,I don't know,1.0


In [5]:
with open("./cv.txt") as f:
    cv_text = f.read() 
    print(to_df(parse(lex(cv_text))))

                  title start_ym   end_ym  \
0      Bechelor of Arts   200809   201306   
1   Bachelor of Science   201009   201306   
2   Master's of Science   201509   201711   
3  Doctor of Philosophy   201711  present   

                                               Major  \
0                 Foreign Languages and Literatures    
1                                        Psychology    
2                                        Psychology    
3  Measurement, Evaluation and Research Methodology    

                       Institution  Education  Spanish  English  Linguistics  \
0      National Taiwan University           1      1.0      1.0          1.0   
1      National Taiwan University           1      NaN      NaN          NaN   
2  University of British Columbia           1      NaN      NaN          1.0   
3  University of British Columbia           1      NaN      NaN          NaN   

   Phonetics  ... Item_Response_Theory  Signal_Detection_Theory Psychometrics  \
0        1.